The Google drive directory is mounted using the folder icon on the left side.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


The working directory is changed

In [ ]:
import os
dirpath = os.getcwd()
print("current directory is : " + dirpath)
foldername = os.path.basename(dirpath)
print("Directory name is : " + foldername)
os.chdir("/content/drive/MyDrive/Estructura_del_PC/conversiones/video")
dirpath = os.getcwd()
print("current directory is : " + dirpath)
foldername = os.path.basename(dirpath)
print("Directory name is : " + foldername)

current directory is : /content
Directory name is : content
current directory is : /content/drive/MyDrive/Estructura_del_PC/conversiones/video
Directory name is : video


# Configuration

In [ ]:
#@title Form { vertical-output: true }
#@markdown Here is the name of the frame. <br>
#@markdown It must contain a '{N}' what represents the number of the frame in python format<br>
#@markdown Ex. 'frame{N}.jpg'
name_format = "frame{N}.jpg" #@param {type:"string"}
heigh =  80#@param {type:"integer"}
width =  80#@param {type: "integer"}

# Importing libraries

In [ ]:
import numpy as np
import imageio
import matplotlib.pyplot as plt
from PIL import Image

# Reading and writing the first frame data.

In [ ]:
f = open("data_image.txt","w")

im0 = imageio.imread(os.path.join(dirpath, name_format.format(N=0)))
im0 = Image.fromarray(im0)
im0 = im0.resize((width, heigh), Image.ANTIALIAS)           #resizing the image
im0 = np.asarray(im0,'uint8')

print(im0.shape)
red = im0[:,:,0]
print(red.shape)
green = im0[:,:,1]
print(green.shape)
blue = im0[:,:,2]
print(blue.shape)

azul = np.uint16(np.trunc((blue/255.0)*(2**5-1)))
verde = np.uint16(np.trunc((green/255.0)*(2**6-1))*(2**5))
rojo = np.uint16(np.trunc((red/255.0)*(2**5-1))*(2**11))
im0 = np.uint16(rojo+verde+azul)

c=0
for y in range(heigh):
  for x in range(width):
    f.write("\t.word\t"+hex(im0[y, x]+(x*(2**16))+(y*(2**23)))+"\n")
    c += 1

(80, 80, 3)
(80, 80)
(80, 80)
(80, 80)


# reading the other frames and saving the changing pixels.

In [ ]:

for i in range(1, 30):
  # Reading the next image
  im1 = imageio.imread(os.path.join(dirpath, name_format.format(N=i)))
  im1 = Image.fromarray(im1)
  im1 = im1.resize((width, heigh), Image.ANTIALIAS)           #resizing the image
  im1 = np.asarray(im1,'uint8') # Getting a multidimensional array with the pixels data

  # print(im1.shape)
  red = im1[:,:,0] #getting the red color matrix
  # print(red.shape)
  green = im1[:,:,1] # Getting the green color matrix
  # print(green.shape)
  blue = im1[:,:,2] # Getting the blue color matrix
  # print(blue.shape)

  azul = np.uint16(np.trunc((blue/255.0)*(2**5-1)))
  verde = np.uint16(np.trunc((green/255.0)*(2**6-1))*(2**5))
  rojo = np.uint16(np.trunc((red/255.0)*(2**5-1))*(2**11))
  im1 = np.uint16(rojo+verde+azul)  # Creating a single matrix with the pixel color data.

  # Writing the pixel color with the format y+x+color (y: bits(23 to 29), x: bits(22 to 16), color: bits(0 to 15))
  # bits 30 and 31 are 0
  # this is part of what can be improved
  for y in range(heigh):
    for x in range(width):
      if im1[y, x] != im0[y, x]:
        f.write("\t.word\t"+hex(im1[y, x]+(x*(2**16))+(y*(2**23)))+"\n")
        c += 1
        
# This can be improved
  im0 = im1
  if i != 29:
    f.write('\t.word\t'+hex(2**30)+'\n')  #this represents the end of a frame, this is not writed if its the last frame

f.write('\t.word\t'+hex(2**31)) # this represents the end of the animation
f.close()
print('The number of words is: ', c)  #the number of words in the animation
print('The number of bytes in words is: ', c*4)
print('\nIn a normal way that could be at least 96000 words')
print('That means 384000 bytes')

The number of words is:  64104
The number of bytes in words is:  256416

In a normal way that would be at least 96000 words
That means 384000 bytes
